In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [2]:
import joblib
import tensorflow as tf

# Load the model using joblib
filename = "./rf_iris.h5"
clf = joblib.load(filename)

# Extract the decision tree models from the random forest classifier
estimators = clf.estimators_
n_classes = clf.n_classes_

# Define a function to combine the outputs of the decision tree models
def combine_predictions(predictions):
    return tf.reduce_mean(tf.one_hot(tf.squeeze(predictions), depth=n_classes), axis=0)

# Define the input layer
inputs = tf.keras.Input(shape=(4,))

# Define the output layer by combining the predictions from the decision tree models
x = tf.stack([estimator.predict(inputs) for estimator in estimators], axis=1)
outputs = tf.keras.layers.Lambda(combine_predictions)(x)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Convert the model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open("./model.tflite", "wb") as f:
    f.write(tflite_model)


TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [1]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Iris 데이터 로드
iris = load_iris()

# 입력과 출력 데이터 추출
X, y = iris.data, iris.target

# 데이터 전처리
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax')
])

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

# 모델 저장
model.save('./iris_model.h5')

# Tensorflow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델 저장
with open('./iris_model.tflite', 'wb') as f:
    f.write(tflite_model)


2023-02-18 15:59:29.735302: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-18 15:59:29.735609: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Epoch 1/50


2023-02-18 15:59:30.344904: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 15:59:30.520624: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-18 15:59:31.760949: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10889b3a0
2023-02-18 15:59:31.760995: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x10889b3a0


NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall' defined at (most recent call last):
    File "/Users/annakim/miniforge3/envs/tensorflow/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/annakim/miniforge3/envs/tensorflow/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/annakim/miniforge3/envs/tensorflow/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/annakim/miniforge3/envs/tensorflow/lib/python3.9/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/Users/annakim/miniforge3/envs/tensorflow/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 724, in start
      self.io_loop.start()
    File "/Users/annakim/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in s